# 01. Create eval dataset from ground truth

In [ ]:
%run ./00_setup.ipynb

## Create eval dataset

In [ ]:
# Create an empty dataset to the experiment
import mlflow

try:
    eval_dataset = mlflow.genai.datasets.create_dataset(
        name=f"{CATALOG}.{SCHEMA}.{EVAL_TABLE}"
    )
    print(f"Created evaluation dataset: {CATALOG}.{SCHEMA}.{EVAL_TABLE}")
except Exception as e:
    if "TABLE_ALREADY_EXISTS" in str(e):
        print(f"Dataset table already exists: {CATALOG}.{SCHEMA}.{EVAL_TABLE}")
        eval_dataset = mlflow.genai.datasets.get_dataset(
            name=f"{CATALOG}.{SCHEMA}.{EVAL_TABLE}",
        )
    else:
        raise e

## Load ground truth data

In [ ]:
# Check ground truth dataset
lease_docs_short_df = spark.read.table(f"{CATALOG}.{SCHEMA}.{GT_TABLE}")
lease_docs_pandas_df = lease_docs_short_df.toPandas()
display(lease_docs_pandas_df.head(1))

## Add ground truth records to eval dataset

In [ ]:
# Convert and add records to eval dataset
records = []
for _, row in lease_docs_pandas_df.iterrows():
    record = {
        "inputs": {"query": row.get("request", "")},
        "expectations": {"expected_response": row.get("labels", "")},
    }
    records.append(record)

records[0]

In [0]:
# Add records to dataset
eval_dataset.merge_records(records)

In [ ]:
# Review the dataset
eval_dataset.to_df()